In [1]:
import sys
sys.path.append('C:/Users/Hendrik/PycharmProjects') # go to parent dir

from ActiveCritic.model_src.whole_sequence_model import WholeSequenceActor, WholeSequenceCritic, WholeSequenceModelSetup
from ActiveCritic.model_src.transformer import TransformerModel, CriticTransformer, ModelSetup
import torch as th
from ActiveCritic.tests.test_utils.utils import make_mask_data, make_seq_encoding_data, make_critic_data, make_wsm_setup

ModuleNotFoundError: No module named 'stable_baselines3'

In [1]:
import torch

In [3]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.Adam
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = TransformerModel

In [4]:
wsm = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=TransformerModel)

In [5]:
wsa = WholeSequenceActor(wsms=wsm)

In [6]:
input = th.ones([batch_size, seq_len, d_intput], dtype=th.float, device='cuda')

In [7]:
output = wsa.forward(inputs=input)

In [8]:
shape = output.shape
assert shape[0] == batch_size
assert shape[1] == seq_len
assert shape[2] == d_output

In [9]:
inpt_seq, outpt_seq = make_seq_encoding_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken, d_out=d_output)
success = th.ones_like(inpt_seq, dtype=th.bool)

In [13]:
wsa = WholeSequenceActor(wsms=wsm)
data = inpt_seq, outpt_seq, success
for i in range(3000):
    result = wsa.forward(inputs=inpt_seq)
    resp = wsa.optimizer_step(data=data)

In [14]:
res = wsa.optimizer_step(data=data)
assert res['Trajectory Loss '] < 1e-2

res = wsa.optimizer_step(data=data, prefix='test')
assert 'Trajectory Loss test' in res


In [15]:
wsa.init_model()

In [16]:
data = inpt_seq, outpt_seq, success
for i in range(1000):
    result = wsa.forward(inputs=inpt_seq)
    wsa.optimizer_step(data=data)

In [17]:
res = wsa.optimizer_step(data=data)
assert res['Trajectory Loss '] < 1e-2

In [19]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_result = 1
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.d_result = d_result
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.Adam
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = CriticTransformer

In [20]:
wsc = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)

In [21]:
inpt_seq, outpt_seq = make_critic_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken)
data = inpt_seq, None, outpt_seq
model = WholeSequenceCritic(wsc)
for i in range(3000):
    res = model.optimizer_step(data)

In [22]:
res

{'critic loss': tensor(1.4034e-12, device='cuda:0'),
 'critic loss positive': tensor(1.4211e-12, device='cuda:0'),
 'critic loss negative': tensor(1.3858e-12, device='cuda:0')}

In [23]:
model.init_model()

In [24]:
res = model.optimizer_step(data)

In [25]:
res

{'critic loss': tensor(0.3448, device='cuda:0'),
 'critic loss positive': tensor(0.6702, device='cuda:0'),
 'critic loss negative': tensor(0.0193, device='cuda:0')}

In [13]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.AdamW
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = TransformerModel
wsa = WholeSequenceActor(wsms=wsm)
input = th.ones([batch_size, seq_len, d_intput], dtype=th.float, device='cuda')
output = wsa.forward(inputs=input)
shape = output.shape

assert(shape[0] == batch_size)
assert(shape[1] == seq_len)
assert(shape[2] == d_output)

inpt_seq, outpt_seq = make_seq_encoding_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken, d_out=d_output)
success = th.ones_like(inpt_seq, dtype=th.bool)
wsa = WholeSequenceActor(wsms=wsm)
data = inpt_seq, outpt_seq, success
for i in range(1000):
    res = wsa.optimizer_step(data=data)
assert(res['Trajectory Loss '] < 1e-2)

res = wsa.optimizer_step(data=data, prefix='test')
assert('Trajectory Loss test' in res)

wsa.init_model()
res = wsa.optimizer_step(data=data)
assert(res['Trajectory Loss '] > 1e-1, 'Init Model did not cange the parameters.')
for i in range(1000):
    res = wsa.optimizer_step(data=data)   
assert(res['Trajectory Loss '] < 1e-2, 'Did not converge after reinit.')

<>:42: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:45: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:42: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:45: SyntaxWarning: assertion is always true, perhaps remove parentheses?


0.001
0.001
0.001


/tmp/ipykernel_9577/2482493908.py:42: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(res['Trajectory Loss '] > 1e-1, 'Init Model did not cange the parameters.')
/tmp/ipykernel_9577/2482493908.py:45: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(res['Trajectory Loss '] < 1e-2, 'Did not converge after reinit.')


In [14]:
res

{'Trajectory Loss ': tensor(2.4412e-09, device='cuda:0')}